In [ ]:
from functions.streamlitfunc import *
from functions.utils import *
import time
import streamlit as st
from functions.chat_bot import *
from functions.utils import local_settings
from streamlit_extras.switch_page_button import switch_page 
# Gather the information provided by the user fill them in the following functions:
# - get_recommendation
# Use the output of the function and present the recommendation in the following format:
# - Restaurant Name
# - Cuisine Type
# - Average Price
# - Location
# - Restaurant Style
# - Restaurant Schedule
# Ask if the user would like to see either see another recommendation, proceed with a reservation \
# at the suggested restaurant or know more about the restaurant.


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from prompts_list import *
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from functions.utils import *
from openai import OpenAI

In [ ]:
files = ['data\CP-23Group4 Project Proposal.pdf', 'data\Food Personalities.pdf']
fil = Filomena()
fil.load_documents(files)

In [ ]:
fil.get_completion('[Identification] Hello.', identified=True)

In [ ]:
for message in fil.messages:
    print(message['content'])

In [ ]:
for message in fil.messages:
    if message["role"] == "user":
        avatar = 'https://miro.medium.com/v2/resize:fit:1100/format:webp/1*_ARzR7F_fff_KI14yMKBzw.png'
        print('user')
    else:
        avatar = avatar="https://cdn.discordapp.com/attachments/1150843302644547768/1190661589347602492/1000_F_378272550_xN8H7ZVudgCYWzfuZxRxVS5uFKjzsoMg.jpg?ex=65a29d04&is=65902804&hm=4a84c24f579a1d8ac5493b28f47b50c1dc7aaabc2832cb090bd7e4e95b2ab786&"
            # with st.chat_message(message["role"], avatar=avatar):
            #     st.markdown(message["content"])

In [ ]:
fil.prep_question('Who are the authors of FlavourFlix?', 
                  "Fellow Foodie, I'm Filomena, your virtual assistant from FlavourFlix! We specialize in recommending restaurants based on your preferences and requirements. So, what are you in the mood for today? Let's find the perfect restaurant for you!")

In [ ]:
from prompt_templates import *
query_helper = GPT_Helper(OPENAI_API_KEY=local_settings.OPENAI_API_KEY, system_behavior=prompt_templates['Query Helper System'])

In [ ]:
def prep_question( query, chatbot_message):
    prompt = f"""
            The User Message is:{query}
            The ChatBot Message is:{chatbot_message}
            """

    query_formatted = query_helper.get_completion(prompt)
    return query_formatted

In [17]:
from functions.utils import *
from functions.chat_bot import *

In [18]:
fil = Filomena()
fil.intialize_filomena(files=['data\CP-23Group4 Project Proposal.pdf', 'data\Food Personalities.pdf'])

In [19]:
fil.messages

[{'role': 'assistant',
  'content': "Hello! Welcome to FlavourFlix, your go-to virtual assistant for restaurant recommendations. I'm Filomena, here to help you find the perfect dining experience. How can I assist you today?"}]

In [ ]:
fil.generate_response('Who are the authors of FlavourFlix?')

In [ ]:
fil.generate_response('What is the purpose of FlavourFlix?')

In [20]:
fil.generate_response('What are the three most expensive restaurants in Albufeira?')

"AI: Thank you for your question! I'm currently searching for the three most expensive restaurants in Albufeira. Please bear with me while I gather the information for you."

In [27]:
from langchain.agents import tool
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [28]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [38]:
@tool
def get_restaurant_info(restaurant_name):
    """Gathers all the restaurant information from the FlavourFlix restaurant database, based on the name of the restaurant."""
    restaurant_data = pd.read_csv('data\preprocessed_restaurant_data.csv')
    restaurant_data = restaurant_data[restaurant_data['name'] == restaurant_name]
    result = f"""Name: {restaurant_data["name"].values[0]} \n /
    Cuisine Type: {restaurant_data["cuisine"].values[0]} \n /
    Average Price: {restaurant_data["averagePrice"].values[0]} \n /
    Location: {restaurant_data["location"].values[0]} \n /
    Restaurant Style: {restaurant_data["style"].values[0]} \n /
    Restaurant Schedule: {restaurant_data["schedule"].values[0]}"""
    return result

In [42]:
client_data = pd.read_csv('data\clientData.csv')

In [51]:
restaurant_data.columns

Index(['url', 'name', 'address', 'photo', 'averagePrice', 'chefName1',
       'chefName2', 'chefName3', 'cuisine', 'michelin', 'description',
       'isBookable', 'maxPartySize', 'schedule', 'promotions', 'phone',
       'photo.1', 'ratingValue', 'reviewCount', 'style', 'latitude',
       'longitude', 'location', 'city', 'ambienceRatingSummary',
       'foodRatingSummary', 'serviceRatingSummary', 'paymentAcceptedSummary',
       'outdoor_area', 'current_occupation', 'menu_pre_proc', 'menu_en',
       'menu_pt'],
      dtype='object')

In [62]:
# !python -m spacy download en_core_web_md

In [70]:
from sklearn.metrics.pairwise import cosine_similarity 
def get_data_match(data, word, col_to_match):
    nlp = spacy.load("en_core_web_md")

    word_embedding = nlp(word).vector
    word_embedding = nlp('mexican').vector
    similarities = {}
    for token in list(data[col_to_match].unique()):
        token_embedding = nlp(token).vector
        similarities[token] = cosine_similarity([word_embedding], [token_embedding])[0][0]

    return max(similarities, key=similarities.get)

In [118]:
def filter_schedule(restaurants, dinner_hour = None, lunch_hour = None):
    def contains_time_interval(schedule):
        if schedule in ['Closed', 'Not Available']:
            return False
        schedule_dict = {}
        try:
            schedule_dict = ast.literal_eval(schedule)
        except (SyntaxError, ValueError):
            pass

        for day_schedule in schedule_dict.values():
            if day_schedule == 'Closed':
                continue
            times = day_schedule.split(', ')
            for time in times:
                start, end = map(pd.to_datetime, time.split(' - '))
                if dinner_hour and lunch_hour:
                    if (start <= dinner_start <= end) or (start <= dinner_end <= end) or (start <= lunch_start <= end) or (start <= lunch_end <= end):
                        return True
                elif dinner_hour:
                    if (start <= dinner_start <= end) or (start <= dinner_end <= end):
                        return True
                elif lunch_hour:
                    if (start <= lunch_start <= end) or (start <= lunch_end <= end):
                        return True
                else:
                    return False
        return False

    dinner_start, dinner_end = map(pd.to_datetime, dinner_hour.split(' - '))
    lunch_start, lunch_end = map(pd.to_datetime, lunch_hour.split(' - '))

    filtered_restaurants = []
    for index, row in restaurants.iterrows():
        if contains_time_interval(row['schedule']):
            filtered_restaurants.append(row['name'])

    return filtered_restaurants

In [127]:
from functions.location import *
from functions.preprocessement import *
from langdetect import detect

def get_recommendation(nationality=None, city=None, travel_car=None,  favourite_food=None,  restaurant_style=None,
                       cuisine_type=None, lunch_hour=None, dinner_hour=None,
                       normal_price_range=None):
    """Recommends restaurants based on the user's preferences and requirements."""
    restaurant_data = pd.read_csv('data/preprocessed_restaurant_data.csv')
    filtered_data = restaurant_data.copy()

    conditions = []

    #Location filters such that the restaurant is near the user and the place the user wants to go
    if city:
        city_match = get_data_match(restaurant_data, city, 'city')
        filtered_data = filtered_data[filtered_data['city'] == city_match]
        latitude, longitude = find_coordinates(city_match)
        if latitude is not None and longitude is not None:
            desired_location = Location(latitude, longitude)
            filtered_data = nearYou(desired_location, filtered_data)
            if travel_car:
                filtered_data['minutes_away'] = filtered_data.apply(lambda row: desired_location.getDirections(row['latitude'], row['longitude'], ['driving'])['driving'].minutes, axis=1)
                filtered_data = filtered_data[filtered_data['minutes_away'] <= 35]
            else:
                filtered_data['minutes_away'] = filtered_data.apply(lambda row: desired_location.getDirections(row['latitude'], row['longitude'], ['walking'])['walking'].minutes, axis=1)
                filtered_data = filtered_data[filtered_data['minutes_away'] <= 59]

    #Time filters such that the restaurant is open at the time the user wants to go
    if lunch_hour:
        filtered_data = filter_schedule(filtered_data, dinner_hour, lunch_hour)
    if dinner_hour:
        filtered_data = filter_schedule(filtered_data, dinner_hour, lunch_hour)
    
    #Additional filters
    if nationality and len(filtered_data) != 0:
        nationality_match = get_data_match(restaurant_data, nationality, 'cuisine')
        conditions.append(restaurant_data['cuisine'] == nationality_match)
    
    if restaurant_style:
        restaurant_style_match = get_data_match(restaurant_data, restaurant_style, 'style')
        conditions.append(restaurant_data['style'] == restaurant_style_match)
    
    if cuisine_type:
        cuisine_type_match = get_data_match(restaurant_data, cuisine_type, 'cuisine')
        conditions.append(restaurant_data['cuisine'] == cuisine_type_match)

    if normal_price_range:
        conditions.append(restaurant_data['averagePrice'] <= normal_price_range+4)

    if favourite_food:
        if detect(favourite_food) == 'en':
            conditions.append(restaurant_data['menu_en'].str.contains(favourite_food))
        else:
            conditions.append(restaurant_data['menu_pre_proc'].str.contains(favourite_food))
    

    # Combine conditions with logical OR
    if conditions and len(filtered_data) != 0:
        filtered_data = restaurant_data[pd.concat(conditions, axis=1).any(axis=1)]
    else:
        filtered_data = restaurant_data 
    

    # Ranking - sort by ratings or other criteria
    restaurant_data = restaurant_data.sort_values(by='ratingValue', ascending=False)

    # Return top recommendations
    return restaurant_data.head(5) 


In [128]:
get_recommendation(city='Albufeira', normal_price_range=20, favourite_food='massa')

,url,name,address,photo,averagePrice,chefName1,chefName2,chefName3,cuisine,michelin,...,city,ambienceRatingSummary,foodRatingSummary,serviceRatingSummary,paymentAcceptedSummary,outdoor_area,current_occupation,menu_pre_proc,menu_en,menu_pt
1324,https://www.thefork.com/restaurant/rei-da-gula...,Rei da Gula,"R. de Ceuta 63,Porto, Portugal",https://res.cloudinary.com/tf-lab/image/upload...,15,Not Applicable,Not Applicable,Not Applicable,Pizzeria,0,...,Porto,NaN,NaN,NaN,"{'Credit Card', 'MBWay'}",0,0,{'Entradas - para começar ': {'Sopa': {'price'...,{'Starters - to get started': {'Sopa': {'price...,{'Entradas - para começar ': {'Soup': {'price'...
224,https://www.thefork.com/restaurant/caravela-wy...,Caravela - Wyndham Grand Algarve,"Av. André Jordan 39,Almancil, Portugal",https://res.cloudinary.com/tf-lab/image/upload...,21,Not Applicable,Not Applicable,Not Applicable,International,0,...,Faro,10.0,10.0,10.0,"{'Credit Card', 'MBWay'}",0,23,{'Entradas': {'Creme De Couve-Flor Com Aroma D...,{'Appetizer': {'Cauliflower Cream with Wasabi ...,{'Starters': {'Creme De Couve-Flor Com Aroma D...
372,https://www.thefork.com/restaurant/alegre-me-h...,Alegre-me Healthy Brunch,"R. Antão Girão 77,Setúbal, Portugal",https://res.cloudinary.com/tf-lab/image/upload...,12,Not Applicable,Not Applicable,Not Applicable,Vegetarian,0,...,Setúbal,10.0,10.0,10.0,{'Credit Card'},0,0,"{'Quesadilhas': {'Surfie': {'price': 9.5, 'des...","{'Quesadillas': {'Surfie': {'price': 9.5, 'des...","{'Quesadilhas': {'Surfista': {'price': 9.5, 'd..."
217,https://www.thefork.com/restaurant/grandbar-lo...,Grandbar & Lounge - Wyndham Grand Algarve,"Av. André Jordan 39,Almancil, Portugal",https://res.cloudinary.com/tf-lab/image/upload...,14,Not Applicable,Not Applicable,Not Applicable,European,0,...,Faro,10.0,10.0,10.0,"{'Credit Card', 'MBWay'}",0,0,{'Bites wga': {'Mini espetadas frango \\\\Yaki...,{'Bites wga': {'Mini chicken kebabs \\\\Yakito...,{'Mordidas wga': {'Mini espetadas frango \\\\Y...
419,https://www.thefork.com/restaurant/casa-do-arc...,Casa do Arco by Douro Exclusive,"5085-252,São Cristovão do Douro, Portugal",https://res.cloudinary.com/tf-lab/image/upload...,35,Pedro Pires,Not Applicable,Not Applicable,European,0,...,Vila Real,9.9,9.6,10.0,"{'Mastercard', 'Credit Card', 'Visa', 'Visa El...",0,0,{'Para Começar': {'Torradas De Pão': {'price':...,"{'To begin': {'Bread Toast': {'price': 8.0, 'd...",{'Para Começar': {'Torradas De Pão': {'price':...


In [33]:
restaurant_data = pd.read_csv('data\preprocessed_restaurant_data.csv')

In [98]:
import pandas as pd
import ast

def filter_schedule(restaurants, dinner_hour, lunch_hour):
    def contains_time_interval(schedule):
        if schedule in ['Closed', 'Not Available']:
            return False
        schedule_dict = {}
        try:
            schedule_dict = ast.literal_eval(schedule)
        except (SyntaxError, ValueError):
            pass

        for day_schedule in schedule_dict.values():
            if day_schedule == 'Closed':
                continue
            times = day_schedule.split(', ')
            for time in times:
                start, end = map(pd.to_datetime, time.split(' - '))
                if (start <= dinner_start <= end) or (start <= dinner_end <= end) or (start <= lunch_start <= end) or (start <= lunch_end <= end):
                    return True
        return False

    dinner_start, dinner_end = map(pd.to_datetime, dinner_hour.split(' - '))
    lunch_start, lunch_end = map(pd.to_datetime, lunch_hour.split(' - '))

    filtered_restaurants = []
    for index, row in restaurants.iterrows():
        if contains_time_interval(row['schedule']):
            filtered_restaurants.append(row['name'])

    return filtered_restaurants


In [54]:
llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo')
agent= initialize_agent( [get_restaurant_info, get_recommendation],
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = False)

In [55]:
result = agent("Tell me about restaurant 'Chutneys' in Albufeira")

In [56]:
result

{'input': "Tell me abour restaurant 'Chutneys' in Albufeira",
 'output': "The restaurant 'Chutneys' in Albufeira is an Indian restaurant with an average price of 14. It is located in Albufeira and has a restaurant style of Festivities. The restaurant is open from 11:30 AM to 2:30 PM and from 6:00 PM to 11:59 PM every day of the week."}

In [31]:
try:
    result = agent("Tell me abour restaurant 'A Galinha da Vizinha' in Albufeira")
except:
    print("exception on external access")



> Entering new AgentExecutor chain...
Question: What is the information about the restaurant 'A Galinha da Vizinha' in Albufeira?
Thought: I should use the get_restaurant_info tool to gather the information about the restaurant.
Action:
```
{
  "action": "get_restaurant_info",
  "action_input": "A Galinha da Vizinha"
}
```
exception on external access


In [134]:
from functions.location import *
from functions.preprocessement import *
from langdetect import detect
from langchain import hub
from sklearn.metrics.pairwise import cosine_similarity 
import spacy
from langchain.agents import tool
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

@tool
def get_restaurant_info(restaurant_name):
    """Gathers all the restaurant information from the FlavourFlix restaurant database, based on the name of the restaurant."""
    restaurant_data = pd.read_csv('data\preprocessed_restaurant_data.csv')
    restaurant_data = restaurant_data[restaurant_data['name'] == restaurant_name]
    result = f"""Name: {restaurant_data["name"].values[0]} \n /
    Cuisine Type: {restaurant_data["cuisine"].values[0]} \n /
    Average Price: {restaurant_data["averagePrice"].values[0]} \n /
    Location: {restaurant_data["location"].values[0]} \n /
    Restaurant Style: {restaurant_data["style"].values[0]} \n /
    Restaurant Schedule: {restaurant_data["schedule"].values[0]}"""
    return result


def get_data_match(data, word, col_to_match):
    nlp = spacy.load("en_core_web_md")

    word_embedding = nlp(word).vector
    word_embedding = nlp('mexican').vector
    similarities = {}
    for token in list(data[col_to_match].unique()):
        token_embedding = nlp(token).vector
        similarities[token] = cosine_similarity([word_embedding], [token_embedding])[0][0]

    return max(similarities, key=similarities.get)


def filter_schedule(restaurants, dinner_hour = None, lunch_hour = None):
    def contains_time_interval(schedule):
        if schedule in ['Closed', 'Not Available']:
            return False
        schedule_dict = {}
        try:
            schedule_dict = ast.literal_eval(schedule)
        except (SyntaxError, ValueError):
            pass

        for day_schedule in schedule_dict.values():
            if day_schedule == 'Closed':
                continue
            times = day_schedule.split(', ')
            for time in times:
                start, end = map(pd.to_datetime, time.split(' - '))
                if dinner_hour and lunch_hour:
                    if (start <= dinner_start <= end) or (start <= dinner_end <= end) or (start <= lunch_start <= end) or (start <= lunch_end <= end):
                        return True
                elif dinner_hour:
                    if (start <= dinner_start <= end) or (start <= dinner_end <= end):
                        return True
                elif lunch_hour:
                    if (start <= lunch_start <= end) or (start <= lunch_end <= end):
                        return True
                else:
                    return False
        return False

    dinner_start, dinner_end = map(pd.to_datetime, dinner_hour.split(' - '))
    lunch_start, lunch_end = map(pd.to_datetime, lunch_hour.split(' - '))

    filtered_restaurants = []
    for index, row in restaurants.iterrows():
        if contains_time_interval(row['schedule']):
            filtered_restaurants.append(row['name'])

    return filtered_restaurants

@tool
def get_recommendation(nationality=None, city=None, travel_car=None,  favourite_food=None,  restaurant_style=None,
                       cuisine_type=None, lunch_hour=None, dinner_hour=None,
                       normal_price_range=None):
    """Recommends restaurants based on the user's preferences and requirements."""
    restaurant_data = pd.read_csv('data/preprocessed_restaurant_data.csv')
    filtered_data = restaurant_data.copy()

    conditions = []

    #Location filters such that the restaurant is near the user and the place the user wants to go
    if city:
        city_match = get_data_match(restaurant_data, city, 'city')
        filtered_data = filtered_data[filtered_data['city'] == city_match]
        latitude, longitude = find_coordinates(city_match)
        if latitude is not None and longitude is not None:
            desired_location = Location(latitude, longitude)
            filtered_data = nearYou(desired_location, filtered_data)
            if travel_car:
                filtered_data['minutes_away'] = filtered_data.apply(lambda row: desired_location.getDirections(row['latitude'], row['longitude'], ['driving'])['driving'].minutes, axis=1)
                filtered_data = filtered_data[filtered_data['minutes_away'] <= 35]
            else:
                filtered_data['minutes_away'] = filtered_data.apply(lambda row: desired_location.getDirections(row['latitude'], row['longitude'], ['walking'])['walking'].minutes, axis=1)
                filtered_data = filtered_data[filtered_data['minutes_away'] <= 59]

    #Time filters such that the restaurant is open at the time the user wants to go
    if lunch_hour:
        filtered_data = filter_schedule(filtered_data, dinner_hour, lunch_hour)
    if dinner_hour:
        filtered_data = filter_schedule(filtered_data, dinner_hour, lunch_hour)
    
    #Additional filters
    if nationality and len(filtered_data) != 0:
        nationality_match = get_data_match(restaurant_data, nationality, 'cuisine')
        conditions.append(restaurant_data['cuisine'] == nationality_match)
    
    if restaurant_style:
        restaurant_style_match = get_data_match(restaurant_data, restaurant_style, 'style')
        conditions.append(restaurant_data['style'] == restaurant_style_match)
    
    if cuisine_type:
        cuisine_type_match = get_data_match(restaurant_data, cuisine_type, 'cuisine')
        conditions.append(restaurant_data['cuisine'] == cuisine_type_match)

    if normal_price_range:
        conditions.append(restaurant_data['averagePrice'] <= normal_price_range+4)

    if favourite_food:
        if detect(favourite_food) == 'en':
            conditions.append(restaurant_data['menu_en'].str.contains(favourite_food))
        else:
            conditions.append(restaurant_data['menu_pre_proc'].str.contains(favourite_food))
    

    # Combine conditions with logical OR
    if conditions and len(filtered_data) != 0:
        filtered_data = restaurant_data[pd.concat(conditions, axis=1).any(axis=1)]
    else:
        filtered_data = restaurant_data 
    

    # Ranking - sort by ratings or other criteria
    restaurant_data = restaurant_data.sort_values(by='ratingValue', ascending=False)

    # Return top recommendations
    return restaurant_data.head(5) 


llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo')
# agent= initialize_agent( [get_restaurant_info],
#     llm,
#     agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
#     handle_parsing_errors=True,
#     verbose = False)

# tools = [get_restaurant_info, get_recommendation]
# # Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/openai-tools-agent")
# agent = create_openai_tools_agent(llm, tools, prompt)
# agent_executor =  AgentExecutor(agent=agent, tools=tools, verbose=True)

# result = agent.invoke({'input': "Tell me about restaurant 'Chutneys' in Albufeira"})
result = agent("Tell me about restaurant 'Chutneys' in Albufeira")
print(result)

{'input': "Tell me about restaurant 'Chutneys' in Albufeira", 'output': "The restaurant 'Chutneys' in Albufeira is an Indian restaurant with an average price of 14. It is located in Albufeira and has a restaurant style of Festivities. The restaurant is open from Monday to Sunday, with the schedule of 11:30 - 14:30 and 18:00 - 23:59."}


In [137]:
from functions.chat_bot_v2 import *

NameError: name 'prompt_templates2' is not defined